In [4]:
import os
import pandas as pd
from google.cloud import bigquery

In [2]:
query_dir = 'queries'

In [5]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [6]:
query_client = bigquery.Client()

In [8]:
request = queries['combined_analysis_data.sql']

In [9]:
real_estate_data = query_client.query(request).to_dataframe()

In [12]:
real_estate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854 entries, 0 to 1853
Data columns (total 18 columns):
ad_id                1854 non-null int64
apt_id               1032 non-null object
new_building         1854 non-null bool
num_bedrooms         1801 non-null float64
floor                1448 non-null float64
primary_size         1643 non-null float64
total_size           1758 non-null float64
price                1774 non-null float64
property_type        1854 non-null object
ownership_type       1854 non-null object
construction_year    1851 non-null float64
energy_character     786 non-null object
common_expenses      587 non-null float64
common_wealth        436 non-null float64
common_debt          575 non-null float64
time_s               1851 non-null float64
short_description    1854 non-null object
full_description     1854 non-null object
dtypes: bool(1), float64(10), int64(1), object(6)
memory usage: 248.1+ KB


In [10]:
real_estate_data.head()

,ad_id,apt_id,new_building,num_bedrooms,floor,primary_size,total_size,price,property_type,ownership_type,construction_year,energy_character,common_expenses,common_wealth,common_debt,time_s,short_description,full_description
0,85287718,120,True,2.0,1.0,48.0,59.0,4150000.0,Leilighet,Eier (Selveier),2019.0,None,NaN,NaN,NaN,905.0,Hovinenga Hus C - nye leiligheter for salg!,Ansvarlig meglerJenny Jakobsen SperstadProsjek...
1,85287718,143,True,2.0,2.0,56.0,61.0,4350000.0,Leilighet,Eier (Selveier),2019.0,None,NaN,NaN,NaN,905.0,Hovinenga Hus C - nye leiligheter for salg!,Ansvarlig meglerJenny Jakobsen SperstadProsjek...
2,85287718,183,True,2.0,5.0,51.0,59.0,4350000.0,Leilighet,Eier (Selveier),2019.0,None,NaN,NaN,NaN,905.0,Hovinenga Hus C - nye leiligheter for salg!,Ansvarlig meglerJenny Jakobsen SperstadProsjek...
3,85287718,187,True,2.0,5.0,48.0,59.0,4350000.0,Leilighet,Eier (Selveier),2019.0,None,NaN,NaN,NaN,905.0,Hovinenga Hus C - nye leiligheter for salg!,Ansvarlig meglerJenny Jakobsen SperstadProsjek...
4,85287718,204,True,3.0,6.0,86.0,92.0,8495000.0,Leilighet,Eier (Selveier),2019.0,None,NaN,NaN,NaN,905.0,Hovinenga Hus C - nye leiligheter for salg!,Ansvarlig meglerJenny Jakobsen SperstadProsjek...
